In [14]:
import numpy as np
import argparse
import cv2
from imutils.video import VideoStream
from imutils.video import FPS
import time
import imutils
from scipy.spatial import distance
from typing import NamedTuple
import collections

In [15]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [16]:
net = cv2.dnn.readNetFromCaffe('SingleShotDetector/MobileNetSSD_deploy.prototxt.txt', 'SingleShotDetector/MobileNetSSD_deploy.caffemodel')

In [17]:
class Object(NamedTuple):
    name: str
    centroidx: float
    centroidy: float
    startX: float
    startY: float
    endX: float
    endY: float
    dissapeared: int
        
    def setDissapeared(self, disp):
        dissapeared = disp

In [18]:
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
fps = FPS().start()

[INFO] starting video stream...


In [19]:
def getBoundingBoxes(detections):

    new_objects = []

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):

        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.8:
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            
            x_centre = (box.astype("int")[0] + box.astype("int")[2]/2).astype("int")
            y_centre = (box.astype("int")[1] + box.astype("int")[3]/2).astype("int")   

            if y_centre >= 300:
                y_centre = 299
            if x_centre >= 400:
                x_centre = 399
            new_objects.append(Object(CLASSES[idx], x_centre, y_centre, startX, startY, endX, endY,0))
    return new_objects

In [20]:
def trackAllObjects(objectId,old_objects,new_objects):
    for x in range(len(new_objects)):
        old_objects[objectId] = new_objects[x]
        objectId += 1
    return objectId,old_objects,new_objects

In [21]:
def extractCentroids(old_objects, new_objects):
    old_centroids = []
    for x in old_objects:
        old_centroids.append([old_objects[x][1],old_objects[x][2]])

    new_centroids = []
    for x in new_objects:
        new_centroids.append([x[1],x[2]])
        
    return old_centroids, new_centroids

In [22]:
def updateExistingObjectLocations(old_idx, new_idx, lst_objectIDs, old_objects, new_objects, dist):
    usedRows = set()
    usedCols = set()

    for (row, col) in zip(old_idx, new_idx):
        if row not in usedRows and col not in usedCols:
            obj_id = lst_objectIDs[row]
            old_objects[obj_id] = new_objects[col]
            old_objects[obj_id] = old_objects[obj_id]._replace(dissapeared=0)

            usedRows.add(row)
            usedCols.add(col)

    unusedRows = set(range(0, dist.shape[0])).difference(usedRows)
    unusedCols = set(range(0, dist.shape[1])).difference(usedCols)
    
    return old_objects, unusedRows, unusedCols

In [23]:
def dealWithDissapearedObjects(unusedRows, lst_objectIDs, old_objects):
    
    for row in unusedRows:

        obj_id = lst_objectIDs[row]
        dissapearance_val = old_objects[obj_id][7]
        old_objects[obj_id] = old_objects[obj_id]._replace(dissapeared=dissapearance_val + 1)


        if old_objects[obj_id][7] > 50:
            del old_objects[obj_id]
            
    return old_objects
        

In [24]:
def trackNewObjects(unusedCols, old_objects, new_objects, objectId):
    for col in unusedCols:
        old_objects[objectId] = new_objects[col]
        objectId += 1
        
    return old_objects, new_objects, objectId

In [25]:
def trackObjects(objectId,old_objects,new_objects):
    if len(old_objects) == 0 :
        
        objectId,old_objects,new_objects = trackAllObjects(objectId,old_objects,new_objects)
        
    else:
        if len(new_objects) > 0:
            
            lst_objectIDs = list(old_objects.keys())
            
            old_centroids, new_centroids = extractCentroids(old_objects, new_objects)

            dist = distance.cdist(np.asarray(old_centroids), np.asarray(new_centroids))

            old_idx = dist.min(axis=1).argsort()
            new_idx = dist.argmin(axis=1)[old_idx]

            old_objects, unusedRows, unusedCols = updateExistingObjectLocations(old_idx, new_idx, lst_objectIDs, old_objects, new_objects, dist)
            
            if dist.shape[0] >= dist.shape[1]:

                old_objects = dealWithDissapearedObjects(unusedRows, lst_objectIDs, old_objects)


            else:
                
                old_objects, new_objects, objectId = trackNewObjects(unusedCols, old_objects, new_objects, objectId)  
                    
    return objectId,old_objects,new_objects
            

In [26]:
objectId = 0
old_objects = collections.OrderedDict()
while True:
    
    frame = vs.read()

    frame = imutils.resize(frame, width=400)

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    net.setInput(blob)
    detections = net.forward()
    
    new_objects = getBoundingBoxes(detections)
    
    objectId, old_objects,new_objects = trackObjects(objectId,old_objects,new_objects)    
               
                    
    
    for obj in old_objects:
        if old_objects[obj][7] == 0:
            # display the prediction
            label = "{},{}".format(old_objects[obj][0], obj)
            cv2.rectangle(frame, (old_objects[obj][3], old_objects[obj][4]), (old_objects[obj][5], old_objects[obj][6]),
                COLORS[obj], 2)
            y = old_objects[obj][4] - 15 if old_objects[obj][4] - 15 > 15 else old_objects[obj][4] + 15
            cv2.putText(frame, label, (old_objects[obj][3], y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[obj], 2)

    
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    # update the FPS counter
    fps.update()
    
    
    new_centroid = [] 

    
# stop the timer and display FPS information
vs.stop()
fps.stop()
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()


[INFO] approx. FPS: 19.02


In [ ]:
old_objects

In [ ]:
onj._setDissapeared(0)

In [ ]:
old_objects[obj_id] = old_objects[obj_id]._replace(dissapeared=8)

In [ ]:
old_objects[obj_id]

In [ ]:
l = collections.OrderedDict()

In [ ]:
l[0] = "fudger"
l[10] = "flabajaba"

In [ ]:
l.get(0)

In [ ]:
for x in l:
    print(x)
    print(l[x])